In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

In [ ]:
stemmer = LancasterStemmer

In [ ]:
import numpy 
import numpy as np
import tflearn
import tensorflow as tf
import random
import json

In [ ]:
with open("intents.json") as file:
     data = json.load(file)

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

In [ ]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

In [ ]:
     
    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))

In [ ]:
labels = sorted(labels)

In [ ]:
training = []
output = []


In [ ]:
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    
    wrds = [stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    
training = numpy.array(training)
output = np.array(output)


In [ ]:
tf.reset_default_graph()

In [ ]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

In [ ]:
model = tflearn.DNN(net)

model.fit(training, output, n_epoch=2000, batch_size=8, show_metric=True)
model.save("model.tflearn")